In [3]:
!pip install azure-ai-textanalytics==5.2.0
!pip install python-telegram-bot
!pip install pyTelegramBotAPI

import random
import os

from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

import telebot



#Here are all the replicas used within the dialogue bot

introPhrase = "Hello, fellow user! I am just a bot, but I really want to listen about your experience in different videogames! I bet you are experienced, and I'd love to hear your stories. I'm sorry if I repeat some of my questions - I have a short memory... I can ask questions about platformers, shooters, adventure and horror games, puzzles, strategies, RPGs and simulations! What do you want to start the chat from?"

greeting = [
    "Hello! Nice to meet you!",
    "Hi friend! So nice to meet you!",
    "Greetings! Glad to meet you!"
]

ending = [
    "It was nice to talk to you!",
    "I'm looking forward to seeing you!",
    "Hope we'll talk again soon!"
]

triggers = [
    "platformer",
    "shooter",
    "adventure",
    "horror",
    "puzzle",
    "strategy",
    "RPG",
    "simulation"
]

triggerReact = [
    "Sure, let's talk about <c>!",
    "Yes, <c> sounds good to me!",
    "Of course, I like <c>!",
    "Of course, let's talk about <c>!",
    "Sure, <c> sounds pretty interesting!",
    "Yes, <c> is the thing I like!",
    "I'll gladly talk about <c>!",
    "<c> sounds good, I'll be glad to talk about it!",
    "<c> is a nice choice!"
]

generalQuestions = [
    "I know some of the games' genres. I bet you've played a lot of them. Which one is your favourite?",
    "Games sure are interesting to play, aren't they? What can you say about them in general?",
    "There are so many stories the games tell, right? What stories do you find the most interesting?",
    "So many types of gameplay out there, isn't it? Which ones do you prefer over the others?"
]

generalReplies = [
    "I'm sorry, I didn't understand you well. Can you please explain a bit?",
    "That's nice. Can you please tell me more about it?",
    "That's interesting. Can you please tell me more about it?"
]

positiveReact = [
    "If you like something, I like that too! Please tell me more about it",
    "It looks like we have a lot in common. I like what you say too. I'd love to hear more of your opinion",
    "Oh, that's my liking! I can talk about what you say for hours. I'd love to hear more of this"
]

negativeReact = [
    "I don't feel like I dislike what you do. Please tell me what makes you think your way",
    "I'm afraid I don't feel the way you are. Can you please tell me why do you think like this?",
    "I don't have such thoughts about this. Can you please explain your opinion?"
]

sentimentReactPos = [
    "I feel like you're in a good mood",
    "I like hearing positive things",
    "Glad to hear you feeling good"
]

sentimentReactNeg = [
    "I feel like you're in a bad mood",
    "Is something up? 'Cause you tell not so positive things",
    "Are you upset with something? Your text doesn't seem very positive"
]

platformerQuestions = [
    "What, in your opinion, makes a good-designed level?",
    "What platformer game is your personal favourite? Can you tell me about it?",
    "What gameplay aspects do you find the most appealing?",
    "How do you think platformer's graphics impacts the gameplay?",
    "What do you think about powerups? Are they always useful?",
    "What about powerups? How do you think they impact the gameplay itself?",
    "Do you like looking for secrets on the levels?",
    "I'm sure you've encountered a boss at least once. What makes an interesting boss fight for you?",
    "Are you a multiplayer person? Or do you enjoy playing by yourself? Can you please tell me why so?",
    "There were a lot of game mechanics made already. Are there any special ones you'd like to see in the future?"
]

shooterQuestions = [
    "What type of in-game weapons do you prefer? Why do you enjoy it?",
    "When it comes to multiplayer, what gameplay do you prefer - team-based or free-for-all? Can you tell me why?",
    "What is your preferrable gameplay style - long-range sniping or close-quaters combat? Can you please tell me why?",
    "What shooter game is your favourite? What made it stand out to you?",
    "Is the storyline in shooter game important to you? Can you describe why?",
    "If the shooter has a story, how important is it to you compared to gameplay mechanics? Can you please tell me why?",
    "How do you think, what is the most important factor that makes the shooter entertaining?",
    "Are there any map design factors that you think make the shooter more challenging?",
    "Some shooters give you the opportunity for being in randomized in-game situations. Have you ever been to an interesting one? Please tell me about it",
    "What do you think about weapon balancing in shooters? Are there any flaws that stand out to you?"
]

adventureQuestions = [
    "Do you prefer open-world games or more linear story-driven ones? Can you tell me why?",
    "What do you think about puzzles in adventure games? Which puzzles do you remember most and why?",
    "Do you like exploring the game worlds? What world aspects make you want to do so?",
    "What do you think can make a game world immersive? Are there any games that stand out to you by this aspect?",
    "Do you find character development crucial for an interesting adventure game? Why do you think so?",
    "Some games let you choose some options within a dialogue. What do you think about this? How do such dialogues impact your experience?",
    "Game atmosphere is considered very important for the enjoyment. What do you think? Can you share a story when you've felt the game's atmosphere?",
    "What kind of player are you - are you a world-explorer or an action-lover? Maybe both? What drives you to this kind of gameplay?",
    "Adventure games are probably the ones with the largest amount of different mechanics. Are there any mechanics that you enjoy the most? Why so?",
    "What adventure game is your favourite? What makes this game or games stand out?"
]

horrorQuestions = [
    "I believe immersion is crucial for any horror game. What you think makes a horror game truly immersive?",
    "There are a lot of scaring techniques made now. Which ones you think work the best for creating fear?",
    "How do you think, can a horror gameplay exist without the darkness around? Why so?",
    "I think storyline is considered important for a horror game. Do you agree? Maybe there are some narrative techniques that stand out to you?",
    "What do you think makes a monster truly terrifying? Is there any monster that really made you scared? Please tell me about it",
    "I'm sure sound design is the number one need for creating the scary atmosphere. What do you think about this?",
    "Sometimes a game's world itself can tell a huge part of the story. What do you think of this method? Why do you think so?",
    "Which do you prefer - the games that limit your ability to fight back, or the ones that give you more control over combat? What makes you think so?",
    "Which do you think is scarier - physical (body) horror or psychological horror? Can you please explain your opinion?",
    "I believe there are some multiplayer horror games. What do you think of them? Does the presence of other people make the game more or less scary?"
]

puzzleQuestions = [
    "There are a lot of puzzle types - 3-in-a-row, mazes, number games etc. Which ones do you enjoy the most? Can you please tell me why?",
    "Do you prefer puzzles that want you to move the objects in space or to solve problems analytically? Why so?",
    "What do you think about difficulty levels in puzzle games? Should the difficulty increase while playing? Why?",
    "Do you prefer puzzle complexity increasing as you progress or doing more challenging tasks from the very beginning? Why so?",
    "I think there are puzzle games where puzzles have randomly generated elements. What do you think about such puzzles? Can they increase replayability?",
    "What do you think about games that include narrative, or even a story? Do you like them or do you prefer focusing on the puzzles themselves?",
    "How do you think, are the effects like music, sounds and graphics helpful or harmful for puzzle game experience? How important are they?",
    "I believe some puzzle games include multiplayer. How do you feel about this? Do you like the competitive elements in puzzle games? Why?",
    "I bet you have some strategy for solving puzzles, especially challenging ones. Can you please share it with me?",
    "Can a puzzle game somehow force you to drop it? Is there something in these games that irritates you?"
]

strategyQuestions = [
    "I believe there are three main strategy subgenres - turn-based, real-time and grand, or 4X. Which one do you prefer? Can you please tell me why?",
    "Some, if not all, strategy games want you to make decisions. How do you approach these decisions, especially with complex and branching choices?",
    "What do you think about resource management in strategy games? How important is it? How do you manage your resources?",
    "Are there any characteristics or playstyles that make their impact on your choice of a faction / race? What are they? Can you describe them?",
    "Within the \"strategy triangle\" of strong-tenacious-numerous factions / races, which one are or can be you? Why do or would you prefer this playstyle?",
    "What do you think about different victory conditions? Do you prefer games with multiple ways to win or games that have just one? Can you please tell me why?",
    "What, in your opinion, makes a strategy game map effective and interesting? Is it just the resource placement or something else? Why so?",
    "How do you think, is it possible to win a strategy game with the least-resourceful start possible? How?",
    "I wonder how do you approach tech and / or progression trees in these games? Which branch do you evolve first? Please tell me why",
    "Are you a strategy multiplayer person or do you prefer playing against AI in singleplayer? Why so?"
]

rpgQuestions = [
    "I believe a lot of RPGs include open worlds. What do you enjoy most while exploring these worlds? Are there any world elements you like the most?",
    "Is character customization important to you when you're preparing to play a game? If it is, how important is the amount of changable details? Why so?",
    "I think RPG games always have a story in them. What do you mostly turn your attention to - storyline or game mechanics? Can you tell me why?",
    "How important for you are choices? Do you like making choices that have impact on gameplay or story? Why? What choice is the most memorable to you?",
    "How do you think, is detailed world-building important for an RPG? Can you tell me what, in your opinion, makes the world well-built?",
    "What do you think about party system in RPGs? In your opinion, what makes a good party? How do you balance or would balance heroes in your party?",
    "How do you think, is interaction between NPCs important for an RPG, even if it's nearly invisible to the player? Why do you think so?",
    "RPGs are the games with, probably, the greatest amounts of dialogues. Do you prefer games with long dialogues, especially branching ones, or more linear games? Why?",
    "There are a lot of different progression systems in RPGs. Which aspects that can be levelled up you find the most satisfying? Can you tell me why?",
    "Within various different quests, what types of quests do you enjoy the most? Storyline, battle quests, searching for an item, maybe something else? Why so?"
]

simulationQuestions = [
    "As far as I know, simulation games cover a lot of themes, from driving a car to managing an nuclear plant and building a city. Which ones do you like the most? Why?",
    "How important for you is realism in simulations? Do you prefer highly realistic games or the ones that have some amount of creativity? Why so?",
    "How do you think realism and fantastic elements can make a good balance in simulation? Why do you think so?",
    "Speaking of simulations, they usually dive deep into mechanics modelling. How important for you is this depth and complexity of mechanics? Why so?",
    "When there is an economy part in the game, how do you approach its management? Are there any economic aspects that you find very interesting? Can you tell me about it?",
    "Some simulations are very challenging. Do you prefer such challenging games or more calm that give you opportunity to spectate more than act? Why so?",
    "I know some simulations include time-management, sometimes very strict. What do you think of this mechanic? Can you tell me how do you handle it?",
    "Have you experienced a simulation where unexpected events can occur? Did you enjoy this experience? Can you tell me how do you handle these events?",
    "Have you been to multiplayer simulations? What do you think about cooperation in a simulated environment? Why so?",
    "I bet simulations can give different experiences. Can you share a story about your most interesting or unexpected experience?"
]



#Now the needed tokens

BotToken = "your_bot_token"

azure_key = "your_azure_language_service_key"
azure_endpoint = "your_azure_language_service_endpoint"

os.environ['LANGUAGE_KEY'] = azure_key
os.environ['LANGUAGE_ENDPOINT'] = azure_endpoint

Endpoint = os.environ.get("LANGUAGE_ENDPOINT")
Key = os.environ.get("LANGUAGE_KEY")

text_analytics_client = TextAnalyticsClient(endpoint=Endpoint, credential=AzureKeyCredential(Key))


os.environ['theme'] = "empty"
global memory
memory = []

def memoryMonitor():
  if len(memory) > 20:
    memory.pop(0)

def prepareResponce(responcePrev):
  if '<c>' in responcePrev:
    responcePrev = responcePrev.replace('<c>', os.environ.get('theme'))
  return responcePrev

def getSentiment(phrase, text_analytics_client):
  documents = [phrase]
  result = text_analytics_client.analyze_sentiment(documents, show_opinion_mining=True)
  docs = [doc for doc in result if not doc.is_error]
  for idx, doc in enumerate(docs):
    return doc.sentiment

#The most important function - the responce selector

def responceSelector(userInput):
  lowerInput = userInput.lower()
  memory.append(userInput)
  memoryMonitor()

  themeChanged = 0

  responces = []
  answers = []
  reactions = []
  #React to the trigger!
  for item in triggers:
    if item.casefold() in userInput.casefold():
      os.environ['theme'] = item
      themeChanged = 1
      break
  answers = answers + generalQuestions
  reactions = reactions + generalReplies
  #If the theme isn't general, we need to include the questions from that theme
  if not os.environ.get('theme') == "empty":
    #Need a switch case here
    match os.environ.get('theme'):
      case "platformer":
        answers = answers + platformerQuestions
      case "shooter":
        answers = answers + shooterQuestions
      case "adventure":
        answers = answers + adventureQuestions
      case "horror":
        answers = answers + horrorQuestions
      case "puzzle":
        answers = answers + puzzleQuestions
      case "strategy":
        answers = answers + strategyQuestions
      case "RPG":
        answers = answers + rpgQuestions
      case "simulation":
        answers = answers + simulationQuestions
    #Also include the like/dislike check and reaction
    sent = getSentiment(lowerInput, text_analytics_client)
    if sent == "positive":
      reactions = reactions + positiveReact
      reactChance = random.randint(0, 100)
      if reactChance < 75:
        responces.append(random.choice(sentimentReactPos))
    elif sent == "negative":
      reactions = reactions + negativeReact
      reactChance = random.randint(0, 100)
      if reactChance < 75:
        responces.append(random.choice(sentimentReactNeg))
  reactionToGo = random.randint(0, 100)
  reactionLastSymbol = ""
  if themeChanged == 1:
    triggersReady = [i for i in triggerReact if i not in memory]
    trig = random.choice(triggersReady)
    trigReady = prepareResponce(trig)
    responces.append(trigReady)
    memory.append(trig)
    memoryMonitor()
  elif reactionToGo < 50:
    reactionsReady = [i for i in reactions if i not in memory]
    react = random.choice(reactionsReady)
    reactionLastSymbol = react[len(react) - 1]
    responces.append(react)
    memory.append(react)
    memoryMonitor()
  if reactionLastSymbol != "?":
    answers = list(set(answers))
    answersReady = [i for i in answers if i not in memory]
    responce = random.choice(answersReady)
    responceReady = prepareResponce(responce)
    responces.append(responceReady)
    memory.append(responce)
    memoryMonitor()
  return responces


bot = telebot.TeleBot(BotToken)

@bot.message_handler(commands=['start'])
def start(message):
  bot.send_message(message.chat.id, introPhrase)

@bot.message_handler(commands=['greet'])
def greet(message):
  reply = random.choice(greeting)
  memory.append(reply)
  memoryMonitor()
  bot.send_message(message.chat.id, reply)

@bot.message_handler(commands=['bye'])
def bye(message):
  reply = random.choice(ending)
  memory.append(reply)
  memoryMonitor()
  bot.send_message(message.chat.id, reply)
  memory.clear()
  bot.stop_polling()

@bot.message_handler(commands=['help'])
def help(message):
  bot.send_message(message.chat.id, "1. If you want to say hello to me, type in /greet\n2. If you want to say bye and clear my memory, type in /bye\n3. If you've forgotten the command list and you want a reminder, type in /help")

@bot.message_handler(content_types=['text'])
def chat(message):
  replies = responceSelector(message.text)
  for reply in replies:
    bot.send_message(message.chat.id, reply)

bot.polling(none_stop=True)